## Создаём предметно-ориентированного чат-бота

Для создания чат-бота, который может отвечать на вопросы по какой-то конкретной теме или предметной области, используется подход Retrieval-Augmented Generation (RAG).

Для начала установим необходимые библиотеки. Мы будем использовать библиотеку LangChain, поскольку она содержит в себе все необходимые компоненты для создания таких чат-ботов:
* общение с языковыми моделями
* векторная база данных
* эмбеддинги

In [ ]:
%pip install langchain langchain_community yandexcloud gigachat chromadb huggingface_hub sentence_transformers telebot

Мы будем строить бота-консультанта для Школы дизайна ВШЭ. Для построения бота нужна текстовая база знаний, содержащая всю необходимую информацию. В идеале она должны быть разбита не небольшие кусочки, каждый из которых содержит законченный фрагмент информации.

Возьмём текст с сайта ШД, немного причёсанный и разбитый на фрагменты символами `//`:

In [27]:
!wget https://github.com/shwars/ai-for-creatives/raw/refs/heads/main/data/openbac.txt

--2025-02-09 17:14:27--  https://github.com/shwars/ai-for-creatives/raw/refs/heads/main/data/openbac.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/shwars/ai-for-creatives/refs/heads/main/data/openbac.txt [following]
--2025-02-09 17:14:27--  https://raw.githubusercontent.com/shwars/ai-for-creatives/refs/heads/main/data/openbac.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36170 (35K) [text/plain]
Saving to: ‘openbac.txt.1’

openbac.txt.1       100%[===================>]  35.32K  --.-KB/s    in 0.04s   

2025-02-09 17:14:28 (831 KB/s) - ‘openbac.txt.1’ saved [36170/36170]



In [28]:
!head openbac.txt

Онлайн-бакалавриат Школы дизайна НИУ ВШЭ — совместный проект Школы дизайна НИУ ВШЭ и НИУ ВШЭ — Пермь. Это инновационная программа, позволяющая объединить преимущества проектного подхода к дизайн-образованию и дистанционного формата. По окончании онлайн-бакалавриата Школы дизайна студенты получают диплом о высшем образовании очной формы обучения.
\\
Онлайн-бакалавриат Школы дизайна НИУ ВШЭ готовит востребованных профессионалов, которые могут работать как в офисе, так и удалённо, как в российских, так и в международных компаниях, агентствах, проектах. В рамках программы в настоящий момент открыто семь профилей обучения: «Коммуникационный дизайн», «Анимация и иллюстрация», «Гейм-дизайн», «Дизайн и программирование», «Дизайн и продвижение цифрового продукта», «Дизайн среды и интерьера», «3D-постпродакшн».
\\
О ПРОГРАММЕ
Обучение в онлайн-бакалавриате Школы дизайна построено на принципах проектного подхода: с первого занятия каждый студент делает собственный проект, получая навыки работы с 

В качестве языковой модели будем использовать Gigachat, поэтому нам потребуется ключ, который можно получить [по инструкции](https://developers.sber.ru/docs/ru/gigachat/quickstart/ind-using-api).

Импортируем необходимые библиотеки и создадим объекты:
* GPT для вызова Gigachat
* embeddings для вычисления семантических эмбеддингов текста. Для этого будем использовать мультиязычную модель [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large).

> Для вычисления эмбеддингов было бы идеально тоже использовать облачный сервис от Gigachat или Yandex, но Gigachat требует оплату за его использование.

> Модель эмбеддингов будет вычисляться внутри Colab, поэтому если не включить GPU - на индексирование всего текста может потребоваться 2-3 минуты.

In [29]:
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models.gigachat import GigaChat
from google.colab import userdata
import warnings
warnings.filterwarnings('ignore')

gigachat_creds = userdata.get('gigachat_creds')

GPT = GigaChat(
    credentials=gigachat_creds,
    scope="GIGACHAT_API_PERS",
    model="GigaChat",
    streaming=False,
    verify_ssl_certs=False,
)

embeddings = HuggingFaceEmbeddings(
    model_name = "intfloat/multilingual-e5-large"
)


Загрузим документ и разобьем его на фрагменты:

In [30]:
doc = open('openbac.txt',encoding='utf-8').read()
docs = doc.split('\\')
print(f"Всего фрагментов: {len(docs)}")
print(f"Макс длина фрагмента: {max(map(len,docs))}")

Всего фрагментов: 63
Макс длина фрагмента: 1314


Поскольку длина макс фрагмента не превышает 1500 символов (500 токенов), то эти фрагменты можно дополнительно не разбивать на части. С учётом такой длины мы можем добавлять в запрос 3-5 найденных фрагментов текста.

Посмотрим, как работает вычисление эмбеддингов:

In [31]:
res = embeddings.embed_documents(docs[0])
print(f"Длина эмбеддингов: {len(res[0])}")

Длина эмбеддингов: 1024


Для хранения всех документов, проиндексированных по эмбеддингам, будем использовать вектоную базу данных, например, ChromaDB. При добавлении текстов в эту базу они будут автоматически проиндексированы:

In [ ]:
db = Chroma('vecstore',embeddings,'./db')
db.add_texts(docs)

Предположим, мы хотим найти среди документов ответ на вопрос **Сколько баллов ЕГЭ нужно для поступления в школу дизайна?**. Для этого определим объект `retriever`, и передадим ему параметр - количество документов, которые надо найти. После этого поиск сводится к простому вызову `retriever.invoke`

In [33]:
q = "Сколько баллов ЕГЭ нужно для поступления в школу дизайна?"

retriever = db.as_retriever(search_kwargs={"k": 3 })
retriever.invoke(q)


[Document(metadata={}, page_content='\nКак поступить в школу дизайна?\n\nЧтобы поступить в Школу дизайна НИУ ВШЭ, необходимо сдать ЕГЭ по русскому языку и литературе, а также успешно пройти творческий конкурс. Минимальный балл по ЕГЭ для русского языка и литературы составляет 45, однако для некоторых профилей, таких как "дизайн и продвижение цифрового продукта", минимальный балл повышается до 60. Творческий конкурс предполагает онлайн-загрузку заранее подготовленного проекта по выбранному профилю обучения на специальный сайт Школы дизайна. Загрузка проектов начнется в мае и будет доступна до конца официальной подачи документов в бакалавриат НИУ ВШЭ'),
 Document(metadata={}, page_content='\nВступительные испытания.\nКакие экзамены необходимо сдать:\nРусский язык (ЕГЭ), минимальный балл 45 (60 для профиля "дизайн и продвижение цифрового продукта")\nЛитература (ЕГЭ), минимальный балл 45 (60 для профиля "дизайн и продвижение цифрового продукта")\nТворческий конкурс, минимальный балл 45 (60

Реализуем самую главную функцию ответа на вопрос! Она работает следующим образом:

1. Вызываем `retriever` и получаем 3 релевантных фрагмента текста
2. Объединяем их вместе в контекст `context`
3. Передаём GPT-модели промпт, включающий в себя исходный вопрос и контекст. Модель в этом случае сама смотрит на контекст и находит в нём нужную информацию.

In [34]:
def answer(q):
  res = retriever.invoke(q)
  context = '\n'.join(x.page_content for x in res)
  prompt = f"""
  Прочитай следующий текст и используй его при ответе на вопрос далее.
  Используй только информацию из текста. Если в тексте нет ответа на данный вопрос,
  напиши, что не знаешь. Вот текст:\n{context}\nВопрос: {q}"""
  return GPT.invoke(prompt).content

answer(q)

'Для поступления в Школу дизайна НИУ ВШЭ необходимо набрать минимум 45 баллов по каждому из ЕГЭ по русскому языку и литературе, если речь идет об основных программах («Дизайн», «Мода», «Современное искусство»). Однако для профиля «дизайн и продвижение цифрового продукта» минимальный балл повышается до 60.'

А теперь оформим это всё в виде телеграм-бота! Для этого используем библиотеку `telebot`. Чтобы создать бота нам нужно сначала получить специальный токен, пообщавшись в telegram со специальным ботом [@botfather](http://t.me/botfather). Полученный токен сохраните в секретах Google Colab.

Код ниже работает в режиме поллинга - он опрашивает сервера telegram на предмет наличия сообщений, и вызывает функцию `handle_message`, если сообщение пришло. Бот в телеграме будет работать только до тех пор, пока следующая ячейка выполняется.

In [35]:
import telebot

telegram_token = userdata.get('tg_token')

bot = telebot.TeleBot(telegram_token)

# Обработчик команды /start
@bot.message_handler(commands=['start'])
def start(message):
    # Отправляем приветственное сообщение
    bot.send_message(message.chat.id,
                     'Привет, я бот, который знает всё про Школу Дизайна НИУ ВШЭ. Спрашивай!')

# Обработчик для всех входящих сообщений
@bot.message_handler(func=lambda message: True)
def handle_message(message):
    ans = answer(message.text)
    bot.send_message(message.chat.id, ans)

# Запуск бота
print("Бот готов к работе")
bot.polling(none_stop=True)

Бот готов к работе


### Выводы

С помощью langchain, векторной базы данных и LLM мы создали предметно-ориентированного чатбота. Качество работы такого бота будет зависеть от многих факторов:
* Насколько качественно собрана текстовая база знаний
* Насколько атомарна информация, содержащаяся во фрагментах базы знаний
* Насколько хорошо работает поиск по эмбеддингам (существуют и другие методы поиска, которые также учитывают ключевые слова)
* От качества работы LLM